## Feature Engineering Plan

The goal of this notebook is to transform unstructured product descriptions into
structured, analysis-ready features.

The process follows an iterative approach:
1. Explore raw product descriptions to understand common patterns.
2. Define product categories based on textual signals.
3. Extract core attributes such as brand, CPU, RAM, and storage.
4. Produce a feature-enriched dataset for downstream analysis or modeling.

In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv("../data/processed/sales_clean.csv")
df.shape

(2397, 16)

### Step 1 — Explore raw product descriptions

This step focuses on understanding the structure, language, and variability
of the product description field before applying any parsing logic.

In [64]:
df["product_description"].sample(20, random_state=42)


443                       acer i5 10210u 8 ssd 512 mx350
196                           asus 14 pen 5405 4 128 int
1508                        dell i7 8650u 8 ssd256 mx130
1292                        hp ryz 3 4300u 8 ssd480 euro
1263                        asus i3 8130u 8 ssd256 mx110
321        msi bravo ryzen 5 5600h 16 ssd512 rx5500 burd
2380                            hp ryz 5 5500u 16 ssd512
1666                         ps5 slim edition new tehn k
1385                 lenovo v15 i3 1215u 8 ssd256 1шт 9к
1418               microsoft surface go i5 1035 4 ssd256
630     hp i5 7300hq 16 ssd512 gtx 1050 tehn отказ 03.10
1608                             acer a9 9420 8 500 tehn
1165                lenovo ryz 5 5600 16 ssd512 gtx 1650
736                            acer e1 1200 8 ssd240 int
613                                      ps 5 2 joy tehn
433                    lenovo ryzen 5 4500u 8 ssd256 int
2034                               dell i5 8250 8 ssd256
471                    lenovo a

In [65]:
df["product_description"].value_counts().head(10)

product_description
service                                       12
lenovo 14 i3 1115g4 8 ssd128                   7
acer ryz 5 7520u 16 ssd512                     5
сист. i7 4790 16 240                           5
hp 830 g5 14 i5 8 8 ssd128                     4
ps 5 blurey                                    4
acer i3 1115g4 8 ssd256                        3
lenovo i5 1035g1 8 ssd256                      3
lenovo 14 i3 1115g4 8 ssd256                   3
lenovo ryz 5 5600h 16 ssd256 1tb rtx3050ti     3
Name: count, dtype: int64

### Step 2 — Brand-based product classification (baseline)

Brands are extracted from product descriptions and used as the primary signal
to identify laptops 

In [66]:
# List of known brands observed in the data
brands = ["lenovo", "dell", "hp", "asus", "acer", "apple", "macbook", "msi", "samsung", "toshiba", \
    "lg", "huawei", "xiaomi", "microsoft", "gateway", "redmi", "realme"]

# Function to extract brand from product description
def extract_brand(text):
    if not isinstance(text, str):
        return "unknown"
    for brand in brands:
        if text.startswith(brand):
            return brand
    return "unknown"

df["brand"] = df["product_description"].apply(extract_brand)


In [67]:
df["brand"].value_counts()


brand
lenovo       593
unknown      404
hp           352
asus         342
acer         270
dell         235
xiaomi        46
macbook       34
apple         30
huawei        30
msi           26
samsung       11
microsoft      9
redmi          8
toshiba        3
gateway        2
lg             1
realme         1
Name: count, dtype: int64

In [68]:
df[["product_description", "brand"]].sample(20, random_state=42)

,product_description,brand
443,acer i5 10210u 8 ssd 512 mx350,acer
196,asus 14 pen 5405 4 128 int,asus
1508,dell i7 8650u 8 ssd256 mx130,dell
1292,hp ryz 3 4300u 8 ssd480 euro,hp
1263,asus i3 8130u 8 ssd256 mx110,asus
321,msi bravo ryzen 5 5600h 16 ssd512 rx5500 burd,msi
2380,hp ryz 5 5500u 16 ssd512,hp
1666,ps5 slim edition new tehn k,unknown
1385,lenovo v15 i3 1215u 8 ssd256 1шт 9к,lenovo
1418,microsoft surface go i5 1035 4 ssd256,microsoft


In [69]:
# assign category "laptop" based on brand names
df["product_category"] = np.where(
    df["brand"] != "unknown",
    "laptop",
    "other"
)

df["product_category"].value_counts()


product_category
laptop    1993
other      404
Name: count, dtype: int64

In [70]:
df[df["brand"] == "unknown"]["product_description"].sample(20, random_state=42)

546                                   ps 4 slim 1tb tehn
1498                                       ps 4 pro euro
2324                      монітор samsung s27r350fhi 1шт
227                                        часы carnival
305                         geo flex 14 i3 1005 4 ssd120
581       моноблок hp 23-q055no i5 4460 8 1tb rad r7 4gb
944                                       ps4 slim 2 joy
2061                     комп i5 10400 16 ssd512 1шт int
2243                оперативка ноутбук ddr4 85шт 408 грн
1696    джойстики playstation 5 dualsense rozetka 8 1900
2242                                              rx6600
582              моноблок 22 hp ryz 5 4500u 8 ssd256 int
1783                             видеокарта nv rtx3070ti
2330                                         nikon d3300
1141            honor i5 10210 8 ssd512 tehn отказ 04.04
1675                               монитор xiaomi p27fba
1260                     surface go 12 i5 1035g1 4 ssd64
2145                           